# 📚 Module 2: Dynamic Content Scraping with Selenium

**Learn to scrape JavaScript-rendered websites**

In this notebook, you'll learn:
- Launch a browser with Selenium
- Wait for dynamic elements
- Extract data from tables
- Export to JSON/CSV

**Target Website**: [CafeF VNINDEX](https://cafef.vn/du-lieu/Lich-su-giao-dich-vnindex-1.chn) (Stock market data)

⚠️ **Note for Colab**: Colab runs in headless mode, so you won't see the browser.

## 🔧 Setup
Install required packages (run once)

In [ ]:
# Install Chrome and Selenium in Colab
!apt-get update
!apt-get install -y chromium-chromedriver
!pip install selenium pydantic pandas -q

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time

def get_chrome_driver():
    """Get Chrome driver configured for Colab (headless)"""
    options = Options()
    options.add_argument('--headless')  # Run without GUI
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    
    driver = webdriver.Chrome(options=options)
    return driver

print("✅ Driver function ready!")

---
## Step 1: Open a Browser with Selenium

**Goal**: Launch Chrome browser and navigate to CafeF

**Concepts**:
- Selenium WebDriver
- Browser automation basics
- Headless mode for Colab

In [ ]:
print("=" * 70)
print("SELENIUM BROWSER AUTOMATION")
print("=" * 70)
print()

# Setup Chrome driver
print("Setting up Chrome driver...")
driver = get_chrome_driver()

print("✅ Browser launched successfully!\n")

# Navigate to CafeF VNINDEX page
url = "https://cafef.vn/du-lieu/Lich-su-giao-dich-vnindex-1.chn"
print(f"Navigating to: {url}")
driver.get(url)

print("✅ Page loaded!")
print(f"Page title: {driver.title}")

# Clean up
driver.quit()
print("\n✅ Browser closed!")

### 💡 Key Takeaways

- `webdriver.Chrome()` launches Chrome
- `driver.get(url)` navigates to a page
- `driver.quit()` closes the browser

---
## Step 2: Wait for Dynamic Elements

**Goal**: Wait for the data table to load before accessing it

**Concepts**:
- Explicit waits (WebDriverWait)
- Expected conditions
- Why waiting is important for dynamic content

In [ ]:
print("=" * 70)
print("WAITING FOR DYNAMIC CONTENT")
print("=" * 70)
print()

# Setup driver
driver = get_chrome_driver()

try:
    # Navigate
    url = "https://cafef.vn/du-lieu/Lich-su-giao-dich-vnindex-1.chn"
    print(f"Loading: {url}\n")
    driver.get(url)
    
    # Create a wait object (waits up to 20 seconds)
    wait = WebDriverWait(driver, 20)
    
    print("Waiting for table to load...")
    print("(This might take a few seconds)\n")
    
    # Wait for the specific table container using XPath
    table_xpath = '//*[@id="render-table-owner"]'
    
    # Wait until the element is present in the DOM
    table_element = wait.until(
        EC.presence_of_element_located((By.XPATH, table_xpath))
    )
    
    print("✅ Table found!")
    print(f"Element tag: {table_element.tag_name}")
    
    # Give it a bit more time for data to fully render
    time.sleep(2)
    
    # Find all rows in the table
    rows = table_element.find_elements(By.TAG_NAME, "tr")
    print(f"✅ Found {len(rows)} rows in the table\n")
    
    # Print first row as example
    if rows:
        print("=" * 70)
        print("FIRST ROW PREVIEW:")
        print("=" * 70)
        first_row = rows[0]
        cells = first_row.find_elements(By.TAG_NAME, "td")
        print(f"Number of columns: {len(cells)}")
        if cells:
            print(f"First column: {cells[0].text}")
            if len(cells) > 1:
                print(f"Second column: {cells[1].text}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    
finally:
    driver.quit()
    print("\n✅ Browser closed!")

### 💡 Key Takeaways

- `WebDriverWait(driver, timeout)` creates a wait object
- `EC.presence_of_element_located()` waits for element to exist
- Dynamic content needs time to load!

---
## Step 3: Extract Table Data

**Goal**: Extract stock data from the table

**Concepts**:
- XPath selectors
- Table row/cell navigation
- Basic data cleaning

In [ ]:
print("=" * 70)
print("EXTRACTING TABLE DATA WITH XPATH")
print("=" * 70)
print()

# Setup
driver = get_chrome_driver()

try:
    # Navigate and wait
    url = "https://cafef.vn/du-lieu/Lich-su-giao-dich-vnindex-1.chn"
    driver.get(url)
    
    wait = WebDriverWait(driver, 20)
    table_xpath = '//*[@id="render-table-owner"]'
    
    print("Waiting for table...")
    table = wait.until(EC.presence_of_element_located((By.XPATH, table_xpath)))
    time.sleep(2)
    
    # Find all rows
    rows = table.find_elements(By.TAG_NAME, "tr")
    print(f"Found {len(rows)} rows\n")
    
    print("=" * 70)
    print("EXTRACTED DATA (First 5 rows):")
    print("=" * 70)
    
    # Extract data from each row
    for i, row in enumerate(rows[:5], 1):  # First 5 rows only
        cells = row.find_elements(By.TAG_NAME, "td")
        
        if not cells:
            continue
        
        # Extract text from each cell
        cell_data = [cell.text.strip() for cell in cells]
        
        # CafeF table structure:
        # 0: Date, 1: Close, 9: Open, 10: High, 11: Low
        
        print(f"\n--- Row {i} ---")
        if len(cell_data) > 0:
            print(f"Date:        {cell_data[0]}")
        if len(cell_data) > 1:
            print(f"Close Price: {cell_data[1]}")
        if len(cell_data) > 9:
            print(f"Open Price:  {cell_data[9]}")
        if len(cell_data) > 10:
            print(f"High Price:  {cell_data[10]}")
    
    print("\n" + "=" * 70)
    print("✅ Successfully extracted data from table!")
    
except Exception as e:
    print(f"❌ Error: {e}")
    
finally:
    driver.quit()
    print("\n✅ Browser closed!")

---
## Step 4: Complete Crawler with Export

**Goal**: Full crawler with Pydantic validation and CSV/JSON export

**Concepts**:
- Complete scraping workflow
- Pydantic data validation
- Multiple export formats

In [ ]:
from pydantic import BaseModel, Field
import json
import pandas as pd

# Define Pydantic model
class StockData(BaseModel):
    """Represents VNINDEX trading data for a single day"""
    date: str = Field(..., description="Trading date")
    open_price: float = Field(default=0.0, description="Opening price")
    close_price: float = Field(default=0.0, description="Closing price")
    high_price: float = Field(default=0.0, description="Highest price")
    low_price: float = Field(default=0.0, description="Lowest price")

def parse_float(text):
    """Convert text to float, handling various formats"""
    if not text or text == '-':
        return 0.0
    try:
        return float(text.replace(',', ''))
    except ValueError:
        return 0.0

print("✅ Models defined!")

In [ ]:
def scrape_vnindex():
    """Scrape VNINDEX trading data from CafeF"""
    print("=" * 70)
    print("COMPLETE VNINDEX CRAWLER")
    print("=" * 70)
    print()
    
    driver = get_chrome_driver()
    results = []
    
    try:
        url = "https://cafef.vn/du-lieu/Lich-su-giao-dich-vnindex-1.chn"
        print(f"Loading: {url}")
        driver.get(url)
        
        wait = WebDriverWait(driver, 20)
        table_xpath = '//*[@id="render-table-owner"]'
        print("Waiting for data table...")
        
        table = wait.until(EC.presence_of_element_located((By.XPATH, table_xpath)))
        time.sleep(2)
        
        rows = table.find_elements(By.TAG_NAME, "tr")
        print(f"Found {len(rows)} rows")
        print("Extracting data...\n")
        
        for row in rows:
            try:
                cells = row.find_elements(By.TAG_NAME, "td")
                if not cells:
                    continue
                
                cell_texts = [c.text.strip() for c in cells]
                
                date = cell_texts[0] if len(cell_texts) > 0 else ""
                if not date or "Ngày" in date:
                    continue
                
                stock_data = StockData(
                    date=cell_texts[0] if len(cell_texts) > 0 else "",
                    open_price=parse_float(cell_texts[9] if len(cell_texts) > 9 else "0"),
                    close_price=parse_float(cell_texts[1] if len(cell_texts) > 1 else "0"),
                    high_price=parse_float(cell_texts[10] if len(cell_texts) > 10 else "0"),
                    low_price=parse_float(cell_texts[11] if len(cell_texts) > 11 else "0")
                )
                
                results.append(stock_data)
                
            except Exception as e:
                continue
        
        print(f"✅ Successfully extracted {len(results)} trading days\n")
        
    except Exception as e:
        print(f"❌ Error during scraping: {e}")
        
    finally:
        driver.quit()
        print("Browser closed\n")
    
    return results

In [ ]:
# Run the crawler
data = scrape_vnindex()

if data:
    # Convert to DataFrame
    df = pd.DataFrame([d.model_dump() for d in data])
    
    # Save to CSV
    csv_file = "vnindex_data.csv"
    df.to_csv(csv_file, index=False)
    print(f"💾 Saved to: {csv_file}")
    
    # Save to JSON
    json_file = "vnindex_data.json"
    with open(json_file, "w", encoding="utf-8") as f:
        json.dump([d.model_dump() for d in data], f, indent=2)
    print(f"💾 Saved to: {json_file}")
    
    # Print sample
    print("\n" + "=" * 70)
    print("SAMPLE DATA:")
    print("=" * 70)
    print(df.head())
    
    print("\n🎉 Complete! You've built a Selenium web scraper!")
else:
    print("❌ No data extracted!")

---
## 🏆 Exercises

1. **Add more fields**: Extract volume, change percentage
2. **Pagination**: Click "Next" button to get more data
3. **Different index**: Modify for HNX instead of VNINDEX
4. **Data analysis**: Plot the closing prices using matplotlib

In [ ]:
# Exercise: Plot the data
import matplotlib.pyplot as plt

if 'df' in dir() and len(df) > 0:
    plt.figure(figsize=(12, 5))
    plt.plot(df['close_price'].head(20), marker='o')
    plt.title('VNINDEX Close Price (Last 20 days)')
    plt.xlabel('Day')
    plt.ylabel('Price')
    plt.grid(True)
    plt.show()